In [ ]:
import pickle
import sys
from pathlib import Path

import numpy as np
from aiida import load_profile, orm
from aiida_workgraph import WorkGraph, task
from tb_hamiltonian.hamiltonian import TBHamiltonian
from tb_hamiltonian.potentials import PotentialFactory
from tb_hamiltonian.utils import get_structure


In [ ]:
sys.tracebacklimit = None

In [ ]:
_ = load_profile()

# Define parameters

In [ ]:
debug = False
use_mpi = False

nn = 1  # number of nearest neighbors | don't use 0!

input_path = Path("output/BLG/AB/rectangular")

# Structure

## lengths
lx = 50  # length in x direction (Å)
ly = lx / np.sqrt(3)  # length in y direction (Å) keeping the b/a ratio
lz = 10  # length in z direction (Å)
workdir = input_path / f"len_{lx}x{int(ly)}/nn_{nn}"

## or, repetitions
nx = 1  # number of repetitions in x direction
ny = 1  # number of repetitions in y direction
nz = 1  # number of repetitions in z direction
workdir = input_path / f"rep_{nx}x{ny}/nn_{nn}"

workdir.mkdir(parents=True, exist_ok=True)

## distances
distances = [0.0, 1.425, 2.468, 2.850]

# Hamiltonian
hopping_parameters = [0.0, -2.7, 0.0, -0.27]
interlater_coupling = 0.33

# potential
alpha = [1.0, 0.5]
onsite_term = 0.0
potential_type = "null"
potential_params = {
    "amplitude": 1.0,
    # "width": 0.5,
    # "height": 0.5,
}

paths = {
    "input_path": input_path.as_posix(),
    "output_path": workdir.as_posix(),
}

band_params = {
    "high_sym_points": {
        "Γ": (0.00000, 0.00000, 0.00000),
        "P": (0.00000, 0.33333, 0.00000),
        "X": (0.00000, 0.50000, 0.00000),
        "W": (0.50000, 0.50000, 0.00000),
        "Y": (0.50000, 0.00000, 0.00000),
    },
    "k_path": "Γ P X W Y Γ W",
    "points_per_segment": 100,
    "use_sparse_solver": False,
    "sparse_solver_params": {"k": 3, "sigma": 1e-8},
    "use_mpi": use_mpi,
    "fig_filename": "bands.png",
}

In [ ]:
@task.calcfunction()
def define_structure(
    paths: dict,
    repetitions: list = None,
) -> orm.StructureData:
    structure = get_structure(
        Path(paths["input_path"]) / "POSCAR",
        repetitions=repetitions or [1, 1, 1],
    )
    structure.info["label"] = "BLG"
    structure.write(Path(paths["output_path"]).parent / "POSCAR", format="vasp")
    return orm.StructureData(ase=structure)

In [ ]:
@task.calcfunction()
def build_hamiltonian(
    structure: orm.StructureData,
    workdir: str,
    nearest_neighbor: int = 1,
    distances: list = None,
    hopping_parameters: list = None,
    interlater_coupling: float = 0.0,
    use_mpi: bool = False,
) -> orm.SinglefileData:
    H = TBHamiltonian(
        structure=structure.get_ase(),
        nearest_neighbor=nearest_neighbor.value,
        distances=distances.get_list() or [0.0],
        hopping_parameters=hopping_parameters.get_list() or [0.0],
        interlayer_coupling=interlater_coupling.value,
    )
    H.build()
    H.write_to_file(Path(workdir.value), use_mpi=use_mpi.value)
    path = Path(workdir.value) / "H.pkl"
    with path.open(mode="wb") as file:
        pickle.dump(H, file)
    return orm.SinglefileData(path.absolute())

In [ ]:
@task.calcfunction(
    outputs=[
        {"name": "H_file"},
        {"name": "workdir"},
    ]
)
def apply_onsite_term(
    H_file: orm.SinglefileData,
    potential_type: str,
    potential_params: dict,
    workdir: str,
    onsite_term: float = 0.0,
    alpha: list = None,
    use_mpi: bool = False,
) -> dict:
    with H_file.open(mode="rb") as file:
        H: TBHamiltonian = pickle.load(file)
    H_onsite = H.copy()
    potential = PotentialFactory(potential_type.value)
    potential.params = potential_params
    H_onsite.update_onsite_terms(onsite_term.value, potential, alpha.get_list() or [1.0])
    path: Path = Path(workdir.value) / potential_type.value
    if potential_type != "null":
        path /= f"amplitude_{potential_params['amplitude']}"
        if "width" in potential_params:
            path /= f"width_{potential_params['width']}"
        if "height" in potential_params:
            path /= f"height_{potential_params['height']}"
    path.mkdir(parents=True, exist_ok=True)
    H_onsite.write_to_file(path, use_mpi=use_mpi)
    pickle_path = path / "H.pkl"
    with pickle_path.open(mode="wb") as file:
        pickle.dump(H_onsite, file)
    return {
        "H_file": orm.SinglefileData(pickle_path.absolute()),
        "workdir": orm.Str(path.absolute().as_posix()),
    }

In [ ]:
@task.calcfunction()
def get_band_structure(
    H_file: orm.SinglefileData,
    workdir: str,
    band_params: dict,
) -> orm.SinglefileData:
    with H_file.open(mode="rb") as file:
        H: TBHamiltonian = pickle.load(file)
    workdir_path = Path(workdir.value)
    filepath: Path = workdir_path / band_params.get("fig_filename", "bands.png")
    H.plot_bands(**{"workdir": workdir_path, **band_params.get_dict()})
    return orm.SinglefileData(filepath.absolute())

In [ ]:
wg = WorkGraph("2D_potential_band_structure")
wg.add_task(
    define_structure,
    name="define_structure",
    paths=paths,
    repetitions=[nx, ny, nz],
)
wg.add_task(
    build_hamiltonian,
    name="build_hamiltonian",
    structure=wg.tasks["define_structure"].outputs["result"],
    workdir=paths["output_path"],
    nearest_neighbor=nn,
    distances=distances,
    hopping_parameters=hopping_parameters,
    interlater_coupling=interlater_coupling,
)
wg.add_task(
    apply_onsite_term,
    name="apply_onsite_term",
    H_file=wg.tasks["build_hamiltonian"].outputs["result"],
    potential_type=potential_type,
    potential_params=potential_params,
    workdir=paths["output_path"],
    onsite_term=onsite_term,
    alpha=alpha,
)
wg.add_task(
    get_band_structure,
    name="get_band_structure",
    H_file=wg.tasks["apply_onsite_term"].outputs["H_file"],
    workdir=wg.tasks["apply_onsite_term"].outputs["workdir"],
    band_params=band_params,
)
node = wg.run()